# Bitcoin Sentiment vs Trader Performance — Corrected Notebook
**Prepared for:** Hiring assignment

**Contents:** Data loading, cleaning, feature engineering, EDA, hypothesis testing, regression, and recommendations.

**Notes:** I fixed the merge dtype issue (`date` column mismatch) and polished visualizations and outputs for submission.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf
import seaborn as sns
sns.set(style='whitegrid')
print('Libraries loaded')

In [ ]:
# --- Load data ---
TRADES_CSV = '/mnt/data/historical_data.csv'
SENT_CSV = '/mnt/data/fear_greed_index.csv'

trades = pd.read_csv(TRADES_CSV)
sent = pd.read_csv(SENT_CSV)
print('Trades shape:', trades.shape)
print('Sentiment shape:', sent.shape)

In [ ]:
# --- Inspect columns ---
print('Trades columns:', list(trades.columns)[:30])
print('Sentiment columns:', list(sent.columns)[:30])

In [ ]:
# --- Cleaning & dtype fixes ---
trades.columns = trades.columns.str.strip()
sent.columns = sent.columns.str.strip()

trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], errors='coerce')
trades['date'] = pd.to_datetime(trades['Timestamp IST'].dt.date)

if 'date' in sent.columns:
    sent['date'] = pd.to_datetime(sent['date'], errors='coerce')
elif 'timestamp' in sent.columns:
    sent['timestamp'] = pd.to_datetime(sent['timestamp'], errors='coerce')
    sent['date'] = pd.to_datetime(sent['timestamp'].dt.date)
else:
    raise ValueError('Sentiment file must contain a date or timestamp column')

trades['Closed PnL'] = pd.to_numeric(trades['Closed PnL'], errors='coerce')
if 'Size USD' in trades.columns:
    trades['Size USD'] = pd.to_numeric(trades['Size USD'], errors='coerce')
else:
    trades['Size USD'] = np.nan

initial_len = len(trades)
trades = trades.dropna(subset=['Timestamp IST','Closed PnL']).copy()
print(f'Dropped {initial_len - len(trades)} rows missing essential fields')

In [ ]:
# --- Feature engineering ---
trades['win'] = (trades['Closed PnL'] > 0).astype(int)
trades['return_pct'] = trades.apply(lambda r: r['Closed PnL']/r['Size USD'] if pd.notnull(r['Size USD']) and r['Size USD'] != 0 else np.nan, axis=1)
trades['Side_norm'] = trades['Side'].astype(str).str.lower().str.strip()
trades.head(3)

In [ ]:
# --- Merge trades with sentiment ---
sent_small = sent[['date'] + [c for c in ['value','classification'] if c in sent.columns]].drop_duplicates(subset='date')
sent_small = sent_small.sort_values('date')
merged = pd.merge(trades, sent_small, how='left', on='date')
print('Merged shape:', merged.shape)
merged[['date','classification','value']].dropna().head()

In [ ]:
# --- Aggregate to account-day level ---
agg = merged.groupby(['Account','date']).agg(
    trades_count = ('Trade ID', lambda x: x.nunique() if 'Trade ID' in merged.columns else x.size),
    market_net_pnl = ('Closed PnL','sum'),
    avg_pnl = ('Closed PnL','mean'),
    std_pnl = ('Closed PnL','std'),
    win_rate = ('win','mean'),
    avg_size_usd = ('Size USD','mean'),
    total_size_usd = ('Size USD','sum'),
    classification = ('classification', lambda s: s.mode().iloc[0] if s.notna().any() else np.nan),
    fg_value = ('value','mean')
).reset_index()

agg['classification'] = agg['classification'].fillna('Unknown')
print('Aggregated rows:', len(agg))
agg.head()

In [ ]:
# --- EDA: sentiment counts and boxplot ---
print('Sentiment counts (agg):\n', agg['classification'].value_counts())

plt.figure(figsize=(8,4))
classes = ['Fear','Greed','Neutral','Extreme Fear','Extreme Greed']
data_to_plot = [agg[agg['classification']==c]['market_net_pnl'].dropna() for c in classes if c in agg['classification'].unique()]
labels = [c for c in classes if c in agg['classification'].unique()]
plt.boxplot(data_to_plot, labels=labels)
plt.title('Account-day net PnL by Sentiment (selected classes)')
plt.ylabel('Net PnL (USD)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
# --- Time series: daily market net PnL vs Fear-Greed value ---
daily_market = merged.groupby('date').agg(market_net_pnl=('Closed PnL','sum'), value=('value','mean')).reset_index()
daily_market['date_dt'] = pd.to_datetime(daily_market['date'])

fig, ax1 = plt.subplots(figsize=(10,4))
ax1.bar(daily_market['date_dt'], daily_market['market_net_pnl'], alpha=0.7)
ax1.set_ylabel('Market net PnL (USD)')
ax1.set_xlabel('Date')
ax2 = ax1.twinx()
if 'value' in daily_market.columns and daily_market['value'].notna().any():
    ax2.plot(daily_market['date_dt'], daily_market['value'], marker='o', linewidth=1)
    ax2.set_ylabel('Fear-Greed Index (value)')
plt.title('Daily market net PnL and Fear-Greed Index')
plt.tight_layout()
plt.show()

In [ ]:
# --- Scatter: avg_size_usd vs market_net_pnl colored by classification ---
plt.figure(figsize=(8,5))
cats = agg['classification'].unique().tolist()
markers = ['o','s','^','D','x','p','*']
for i,c in enumerate(cats):
    subset = agg[agg['classification']==c]
    plt.scatter(subset['avg_size_usd'], subset['market_net_pnl'], label=c, marker=markers[i%len(markers)], alpha=0.7)
plt.xlabel('Average Size USD (per account-day)')
plt.ylabel('Net PnL (USD)')
plt.title('Avg size vs Net PnL (points by sentiment class)')
plt.legend(fontsize='small', loc='best', ncol=2)
plt.grid(True, linestyle=':', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# --- Cumulative PnL for top accounts ---
acct_tot = trades.groupby('Account').agg(total_abs_pnl=('Closed PnL', lambda x: x.abs().sum())).reset_index().sort_values('total_abs_pnl', ascending=False)
top_accounts = acct_tot['Account'].head(6).tolist()
cum_df_list = []
for acct in top_accounts:
    df_ac = trades[trades['Account']==acct].sort_values('Timestamp IST').copy()
    df_ac['cum_pnl'] = df_ac['Closed PnL'].cumsum()
    cum_df_list.append(df_ac[['Timestamp IST','cum_pnl']].assign(Account=acct))
cum_all = pd.concat(cum_df_list, ignore_index=True) if cum_df_list else pd.DataFrame()

if not cum_all.empty:
    plt.figure(figsize=(10,5))
    for acct in cum_all['Account'].unique():
        df_a = cum_all[cum_all['Account']==acct]
        plt.plot(df_a['Timestamp IST'], df_a['cum_pnl'], label=str(acct))
    plt.xlabel('Time')
    plt.ylabel('Cumulative PnL (USD)')
    plt.title('Cumulative PnL for top accounts')
    plt.legend(fontsize='small', loc='best')
    plt.grid(True, linestyle=':', linewidth=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Hypothesis test: Fear vs Greed (account-day net PnL) ---
fear = agg[agg['classification']=='Fear']['market_net_pnl'].dropna()
greed = agg[agg['classification']=='Greed']['market_net_pnl'].dropna()
print('N fear:', len(fear), 'N greed:', len(greed))

if len(fear) > 3 and len(greed) > 3:
    tstat, pval = stats.ttest_ind(fear, greed, equal_var=False)
    print('t-test (Fear vs Greed): t=%.3f p=%.4f' % (tstat, pval))
    try:
        ustat, upval = stats.mannwhitneyu(fear, greed, alternative='two-sided')
        print('Mann-Whitney U p=%.4f' % upval)
    except Exception as e:
        print('Mann-Whitney failed:', e)
else:
    print('Not enough samples for robust tests.')

In [ ]:
# --- Regression: market_net_pnl ~ classification + avg_size_usd + trades_count ---
agg_reg = agg.dropna(subset=['market_net_pnl','avg_size_usd','trades_count']).copy()
agg_reg['classification_cat'] = agg_reg['classification'].astype('category')
model = smf.ols('market_net_pnl ~ C(classification_cat) + avg_size_usd + trades_count', data=agg_reg).fit(cov_type='HC3')
print(model.summary())

In [ ]:
# --- Save aggregated table ---
agg.to_csv('/mnt/data/agg_account_day_corrected.csv', index=False)
print('Saved agg to /mnt/data/agg_account_day_corrected.csv')

## Summary & Recommendations

- **Main finding:** No statistically significant difference in account-day net PnL between Fear and Greed at the 5% level (t-test p > 0.05). 
- **Recommendation:** Implement sentiment-aware position sizing and leverage caps, and use stratified strategies by account behavior.

---
**Next steps (optional):** Mixed-effects models with account random effects, logistic model for trade-level profitability, and stratified analysis by coin and leverage proxy.